In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44534
2,Huelva,Confirmados PDIA 14 días,1085
3,Huelva,Tasa PDIA 14 días,"211,43090983494747"
4,Huelva,Confirmados PDIA 7 días,395
5,Huelva,Tasa PDIA 7 dias,"76,97254321180115"
6,Huelva,Total Confirmados,44733
7,Huelva,Curados,40162
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44534.0


/tmp/ipykernel_30513/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12362.0


/tmp/ipykernel_30513/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_30513/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_30513/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_30513/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1598 personas en los últimos 7 días 

Un positivo PCR cada 587 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44534.0,395.0,1085.0,513170.0,76.972543,211.430910,17.0
Huelva-Costa,26306.0,238.0,675.0,289548.0,82.197080,233.121969,11.0
Cartaya,1861.0,26.0,81.0,20083.0,129.462730,403.326196,5.0
Huelva (capital),12362.0,90.0,245.0,143837.0,62.570827,170.331695,4.0
Condado-Campiña,13870.0,115.0,297.0,156231.0,73.608951,190.103117,3.0
Sierra de Huelva-Andévalo Central,3942.0,37.0,97.0,67391.0,54.903474,143.936134,2.0
Cerro de Andévalo (El),195.0,6.0,10.0,2327.0,257.842716,429.737860,1.0
Cortegana,290.0,3.0,5.0,4602.0,65.189048,108.648414,1.0
Isla Cristina,3053.0,33.0,95.0,21393.0,154.256065,444.070490,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,30.0,3.0,4.0,396.0,757.575758,1010.101010,0.0
Cabezas Rubias,52.0,1.0,6.0,706.0,141.643059,849.858357,0.0
Rosal de la Frontera,71.0,2.0,13.0,1697.0,117.855038,766.057749,0.0
Sanlúcar de Guadiana,34.0,3.0,3.0,403.0,744.416873,744.416873,0.0
Escacena del Campo,231.0,3.0,15.0,2287.0,131.176213,655.881067,0.0
San Silvestre de Guzmán,21.0,4.0,4.0,614.0,651.465798,651.465798,0.0
San Bartolomé de la Torre,374.0,15.0,20.0,3761.0,398.830098,531.773465,0.0
Higuera de la Sierra,62.0,4.0,6.0,1291.0,309.837335,464.756003,0.0
Lucena del Puerto,362.0,6.0,15.0,3261.0,183.992640,459.981601,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,829.0,1.0,13.0,12750.0,7.843137,101.960784,0.0,0.076923
Bonares,451.0,1.0,8.0,6060.0,16.501650,132.013201,0.0,0.125000
Rosal de la Frontera,71.0,2.0,13.0,1697.0,117.855038,766.057749,0.0,0.153846
Bollullos Par del Condado,1257.0,1.0,6.0,14387.0,6.950719,41.704316,0.0,0.166667
Cabezas Rubias,52.0,1.0,6.0,706.0,141.643059,849.858357,0.0,0.166667
San Juan del Puerto,752.0,4.0,22.0,9411.0,42.503453,233.768994,1.0,0.181818
Escacena del Campo,231.0,3.0,15.0,2287.0,131.176213,655.881067,0.0,0.200000
Beas,356.0,2.0,9.0,4341.0,46.072334,207.325501,0.0,0.222222
Villanueva de los Castillejos,206.0,2.0,9.0,2825.0,70.796460,318.584071,0.0,0.222222
